In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import dfply as dp
import random
import warnings
from sklearn import datasets
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from mlxtend.feature_selection import SequentialFeatureSelector
from mlxtend.plotting import plot_sequential_feature_selection
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.metrics.classification import precision_score, recall_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, confusion_matrix
from keras import Sequential
from sklearn.ensemble import VotingClassifier

C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
data = pd.read_csv(r"C:\Users\Administrator\Downloads\Micro-Credit-Project\Data file.csv")

In [3]:
data.head()

,Unnamed: 0,label,msisdn,aon,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,...,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90,pcircle,pdate
0,1,0,21408I70789,272.0,3055.050000,3065.150000,220.13,260.13,2.0,0.0,...,6.0,0.0,2.0,12,6,0.0,29.000000,29.000000,UPW,2016-07-20
1,2,1,76462I70374,712.0,12122.000000,12124.750000,3691.26,3691.26,20.0,0.0,...,12.0,0.0,1.0,12,12,0.0,0.000000,0.000000,UPW,2016-08-10
2,3,1,17943I70372,535.0,1398.000000,1398.000000,900.13,900.13,3.0,0.0,...,6.0,0.0,1.0,6,6,0.0,0.000000,0.000000,UPW,2016-08-19
3,4,1,55773I70781,241.0,21.228000,21.228000,159.42,159.42,41.0,0.0,...,6.0,0.0,2.0,12,6,0.0,0.000000,0.000000,UPW,2016-06-06
4,5,1,03813I82730,947.0,150.619333,150.619333,1098.90,1098.90,4.0,0.0,...,6.0,0.0,7.0,42,6,0.0,2.333333,2.333333,UPW,2016-06-22


In [4]:
data.dtypes

Unnamed: 0                int64
label                     int64
msisdn                   object
aon                     float64
daily_decr30            float64
daily_decr90            float64
rental30                float64
rental90                float64
last_rech_date_ma       float64
last_rech_date_da       float64
last_rech_amt_ma          int64
cnt_ma_rech30             int64
fr_ma_rech30            float64
sumamnt_ma_rech30       float64
medianamnt_ma_rech30    float64
medianmarechprebal30    float64
cnt_ma_rech90             int64
fr_ma_rech90              int64
sumamnt_ma_rech90         int64
medianamnt_ma_rech90    float64
medianmarechprebal90    float64
cnt_da_rech30           float64
fr_da_rech30            float64
cnt_da_rech90             int64
fr_da_rech90              int64
cnt_loans30               int64
amnt_loans30              int64
maxamnt_loans30         float64
medianamnt_loans30      float64
cnt_loans90             float64
amnt_loans90              int64
maxamnt_

In [5]:
data.shape

(209593, 37)

In [6]:
index_names = data[ data['cnt_loans30'] == 0 ].index
data.drop(index_names, inplace = True)

In [7]:
index_names1 = data[ data['cnt_loans90'] == 0 ].index
data.drop(index_names1,inplace = True)

In [8]:
data.shape

(206334, 37)

In [9]:
data = data.rename(columns = {'Unnamed: 0':'Customer_number'},inplace = False)

In [10]:
data.set_index('Customer_number',inplace = True)

In [11]:
data.skew(axis = 0)

label                   -2.243226
aon                     10.412316
daily_decr30             3.965795
daily_decr90             4.284159
rental30                 4.468993
rental90                 4.379261
last_rech_date_ma       14.802413
last_rech_date_da       14.763021
last_rech_amt_ma         3.838389
cnt_ma_rech30            3.276838
fr_ma_rech30            14.800651
sumamnt_ma_rech30        6.402651
medianamnt_ma_rech30     3.553197
medianmarechprebal30    14.778444
cnt_ma_rech90            3.427664
fr_ma_rech90             2.290585
sumamnt_ma_rech90        4.923172
medianamnt_ma_rech90     3.799376
medianmarechprebal90    45.512392
cnt_da_rech30           17.803363
fr_da_rech30            14.764186
cnt_da_rech90           27.137861
fr_da_rech90            28.775577
cnt_loans30              2.737129
amnt_loans30             2.997192
maxamnt_loans30         17.648609
medianamnt_loans30       4.510679
cnt_loans90             16.573289
amnt_loans90             3.145134
maxamnt_loans9

In [12]:
data = data.drop(['msisdn','pcircle','pdate'],axis = 1)

In [13]:
x_data = data.drop(['label'],axis =1)
y_data = data['label']

In [14]:
from skew_autotransform import skew_autotransform

In [15]:
skew_autotransform(x_data, include = None, exclude = None, plot = False, threshold = 1, exp = False)

,aon,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,last_rech_amt_ma,cnt_ma_rech30,fr_ma_rech30,...,cnt_loans30,amnt_loans30,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90
Customer_number,,,,,,,,,,,,,,,,,,,,,
1,5.007875,7.063366,6.857964,13.727007,11.583752,2.868565,1.986364,31.624181,0.793481,2.993640,...,0.589553,1.551326,0.136433,-1.666328e+06,0.600292,1.700162,0.089546,-2.098241e+07,3.113993,3.483684
2,5.641413,8.076737,7.802499,13.971596,11.751494,3.176564,1.986364,50.836330,0.009968,-4.726050,...,0.000000,1.551326,0.136433,-1.666328e+06,0.000000,1.700162,0.089546,-2.098241e+07,-5.143115,-4.399384
3,5.450142,6.484916,6.310847,13.777442,11.616678,2.890417,1.986364,31.624181,0.009968,-4.726050,...,0.000000,1.264623,0.136433,-1.666328e+06,0.000000,1.355357,0.089546,-2.098241e+07,-5.143115,-4.399384
4,4.931418,4.382941,4.305022,13.722439,11.578496,3.406182,1.986364,26.450913,-2.251392,-4.726050,...,0.589553,1.551326,0.136433,-1.666328e+06,0.600292,1.700162,0.089546,-2.098241e+07,-5.143115,-4.399384
5,5.833189,5.021444,4.918549,13.791938,11.626742,2.911526,1.986364,36.636475,2.819470,0.695411,...,1.264243,1.900247,0.136433,-1.666328e+06,1.323383,2.155489,0.089546,-2.098241e+07,0.834712,0.858870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209589,5.264127,5.025714,4.922639,13.791232,11.626252,2.845917,1.986364,44.791346,1.351915,0.695411,...,0.589553,1.551326,0.136433,-1.666328e+06,0.600292,1.700162,0.089546,-2.098241e+07,0.009948,0.009951
209590,5.918487,4.492695,4.410740,13.837131,11.658119,2.911526,1.986364,24.524225,1.800809,0.009950,...,0.853629,1.684589,0.136433,-1.666328e+06,0.877601,1.868410,0.089546,-2.098241e+07,0.009948,0.009951
209591,5.878522,8.059856,7.790136,14.110120,11.971470,2.890417,1.986364,31.624181,2.182549,2.056625,...,1.012133,1.900247,0.136433,-1.666328e+06,1.253765,2.226409,0.089546,-2.098241e+07,1.344317,1.364637


In [16]:
X = x_data
Y = y_data
size = 0.3

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=size)

In [18]:
def create_pipeline(norm, model):
    if norm == 1:
        scale = StandardScaler()
        pipe = Pipeline([('norm', scale), ('reg', model)])
    elif norm == 2:
        scale = MinMaxScaler()
        pipe = Pipeline([('norm', scale), ('reg', model)])
    else:
        pipe = Pipeline([('reg', model)])
    return pipe

In [19]:
def select_features(model, X_train, Y_train, selection,
                    score_criteria, see_details, norm=0):
    pipe = create_pipeline(norm, model)
    sfs = SequentialFeatureSelector(pipe,
                                    forward=selection,
                                    k_features= 12,
                                    scoring=score_criteria,
                                    verbose=see_details)
    sfs = sfs.fit(X_train, Y_train)
    return list(sfs.k_feature_idx_)

In [20]:
def run_model(model, param_grid, X_train, Y_train,
              X, Y, score_criteria, folds,
              see_details, norm=0):
    pipe = create_pipeline(norm, model)
    model_grid = GridSearchCV(pipe,
                              param_grid,
                              cv=folds,
                              scoring=score_criteria,
                              verbose=see_details)
    model_grid.fit(X_train, Y_train)

    pipe = create_pipeline(norm, model_grid.best_estimator_)
    return model_grid.best_estimator_

In [21]:
def get_model_eval(model, X_train, Y_train, X_test, Y_test):
    cm = confusion_matrix(Y_test, model.predict(X_test))
    t1, f1, t0, f0 = cm[1, 1], cm[1, 0], cm[0, 0], cm[0, 1]
    precision = precision_score(Y_test, model.predict(X_test))
    recall = recall_score(Y_test, model.predict(X_test))
    return pd.Series([model,
                      (t1 + t0) / (t1 + t0 + f1 + f0),
                      precision,
                      recall,
                      2 * precision * recall / (precision + recall),
                      -1 if type(model.steps[1][1]) == RidgeClassifier else roc_auc_score(Y_test, model.predict_proba(X_test)[:, 1])])

In [22]:
PARAM_DICT = {
  LogisticRegression: {
    'reg__tol': [1e-2, 1e-4, 1e-6],
    'reg__fit_intercept': [True, False],
    'reg__penalty': ['l1', 'l2']
  },
  RidgeClassifier: {
    'reg__alpha': [0.1, 1, 100],
    'reg__copy_X': [True, False],
    'reg__fit_intercept': [True, False],
    'reg__tol': [0.1, 1],
    'reg__solver': ['auto', 'svd', 'cholesky', 'lsqr',
      'sparse_cg', 'sag', 'saga'
    ]
  },

  KNeighborsClassifier: {
    'reg__n_neighbors': [5, 30, 100]
  },
  GaussianNB: {
  },

  DecisionTreeClassifier: {
    'reg__max_depth': [5, 10, 20],
    'reg__max_features': [0.3, 0.7, 1.0],
    'reg__max_leaf_nodes': [10, 50, 100],
    'reg__splitter': ['best', 'random']
  },

  BaggingClassifier: {
    'reg__bootstrap': [True, False],
    'reg__bootstrap_features': [True, False],
    'reg__max_features': [0.3, 0.7, 1.0],
    'reg__max_samples': [0.3, 0.7, 1.0],
    'reg__n_estimators': [10, 50, 100]
  },
  RandomForestClassifier: {
    'reg__bootstrap': [True, False],
    'reg__max_depth': [5, 10, 20],
    'reg__max_features': [0.3, 0.7, 1.0],
    'reg__max_leaf_nodes': [10, 50, 100],
    'reg__min_impurity_decrease': [0, 0.1, 0.2],
    'reg__n_estimators': [10, 50, 100]
  },

  SVC: {
    'reg__C': [10 ** -3, 1, 1000],
    'reg__kernel': ['linear', 'poly', 'rbf'],
    'reg__shrinking': [True, False],
    'reg__probability': [True]
  },

  GradientBoostingClassifier: {
    'reg__learning_rate': [0.1, 0.2, 0.5],
    # 'reg__loss': ['ls', 'lad', 'huber', 'quantile'],
    'reg__max_depth': [10, 20, 50],
    'reg__max_features': [0.5, 0.8, 1.0],
    'reg__max_leaf_nodes': [10, 50, 100],
    'reg__min_impurity_decrease': [0, 0.1, 0.2],
    'reg__min_samples_leaf': [5, 10, 20],
    'reg__min_samples_split': [5, 10, 20],
    'reg__n_estimators': [10, 50, 100]
  },
  XGBClassifier: {
    'reg__booster': ['gbtree', 'gblinear', 'dart'],
    'reg__learning_rate': [0.2, 0.5, 0.8],
    'reg__max_depth': [5, 10, 20],
    'reg__n_estimators': [10, 50, 100],
    'reg__reg_alpha': [0.1, 1, 10],
    'reg__reg_lambda': [0.1, 1, 10],
    'reg__subsample': [0.3, 0.5, 0.8],
    'reg__probability': [True]
  }
}

In [23]:
random.seed(100)
norm = 1
folds = 4
score_criteria = 'f1'
see_details = 2

In [24]:
results = pd.DataFrame(columns=['model', 'Accuracy', 'PrecisionLab1', 'RecallLab1',
                                'FMeasureLab1', 'AUC'])

In [25]:
to_run = [DecisionTreeClassifier,
          BaggingClassifier]
          

In [26]:
for model in to_run:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        best_feat = select_features(model(), X_train, Y_train, True,
                                    score_criteria, see_details, norm)
        model = run_model(model(), PARAM_DICT[model],
                          X_train.iloc[:, best_feat],
                          Y_train,
                          X.iloc[:, best_feat], Y,
                          score_criteria, folds, see_details, norm)
        stats = get_model_eval(model, X_train.iloc[:, best_feat], Y_train,
                               X_test.iloc[:, best_feat], Y_test)
        stats.index = results.columns
        results = results.append(stats, ignore_index=True)
print(results)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:   49.4s finished

[2020-09-25 12:36:13] Features: 1/12 -- score: 0.9326322635142127[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  31 out of  31 | elapsed:   49.7s finished

[2020-09-25 12:37:03] Features: 2/12 -- score: 0.9326322635142127[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   52.3s finished

[2020-09-25 12:37:55] Features: 3/12 -- score: 0.9326322635142127[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

Fitting 4 folds for each of 54 candidates, totalling 216 fits
[CV] reg__max_depth=5, reg__max_features=0.3, reg__max_leaf_nodes=10, reg__splitter=best 
[CV]  reg__max_depth=5, reg__max_features=0.3, reg__max_leaf_nodes=10, reg__splitter=best, total=   0.2s
[CV] reg__max_depth=5, reg__max_features=0.3, reg__max_leaf_nodes=10, reg__splitter=best 
[CV]  reg__max_depth=5, reg__max_features=0.3, reg__max_leaf_nodes=10, reg__splitter=best, total=   0.2s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s



[CV] reg__max_depth=5, reg__max_features=0.3, reg__max_leaf_nodes=10, reg__splitter=best 
[CV]  reg__max_depth=5, reg__max_features=0.3, reg__max_leaf_nodes=10, reg__splitter=best, total=   0.2s
[CV] reg__max_depth=5, reg__max_features=0.3, reg__max_leaf_nodes=10, reg__splitter=best 
[CV]  reg__max_depth=5, reg__max_features=0.3, reg__max_leaf_nodes=10, reg__splitter=best, total=   0.2s
[CV] reg__max_depth=5, reg__max_features=0.3, reg__max_leaf_nodes=10, reg__splitter=random 
[CV]  reg__max_depth=5, reg__max_features=0.3, reg__max_leaf_nodes=10, reg__splitter=random, total=   0.1s
[CV] reg__max_depth=5, reg__max_features=0.3, reg__max_leaf_nodes=10, reg__splitter=random 
[CV]  reg__max_depth=5, reg__max_features=0.3, reg__max_leaf_nodes=10, reg__splitter=random, total=   0.1s
[CV] reg__max_depth=5, reg__max_features=0.3, reg__max_leaf_nodes=10, reg__splitter=random 
[CV]  reg__max_depth=5, reg__max_features=0.3, reg__max_leaf_nodes=10, reg__splitter=random, total=   0.1s
[CV] reg__ma

[Parallel(n_jobs=1)]: Done 216 out of 216 | elapsed:   50.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:  4.6min finished

[2020-09-25 12:54:20] Features: 1/12 -- score: 0.9326322635142127[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  31 out of  31 | elapsed:  4.4min finished

[2020-09-25 12:58:44] Features: 2/12 -- score: 0.9326322635142127[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  4.6min finished

[2020-09-25 13:03:18] Features: 3/12 -- score: 0.9326322635142127[Parallel(n_jobs=1)]: Using backend Seq

Fitting 4 folds for each of 108 candidates, totalling 432 fits
[CV] reg__bootstrap=True, reg__bootstrap_features=True, reg__max_features=0.3, reg__max_samples=0.3, reg__n_estimators=10 
[CV]  reg__bootstrap=True, reg__bootstrap_features=True, reg__max_features=0.3, reg__max_samples=0.3, reg__n_estimators=10, total=   0.7s
[CV] reg__bootstrap=True, reg__bootstrap_features=True, reg__max_features=0.3, reg__max_samples=0.3, reg__n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV]  reg__bootstrap=True, reg__bootstrap_features=True, reg__max_features=0.3, reg__max_samples=0.3, reg__n_estimators=10, total=   0.7s
[CV] reg__bootstrap=True, reg__bootstrap_features=True, reg__max_features=0.3, reg__max_samples=0.3, reg__n_estimators=10 
[CV]  reg__bootstrap=True, reg__bootstrap_features=True, reg__max_features=0.3, reg__max_samples=0.3, reg__n_estimators=10, total=   0.8s
[CV] reg__bootstrap=True, reg__bootstrap_features=True, reg__max_features=0.3, reg__max_samples=0.3, reg__n_estimators=10 
[CV]  reg__bootstrap=True, reg__bootstrap_features=True, reg__max_features=0.3, reg__max_samples=0.3, reg__n_estimators=10, total=   0.7s
[CV] reg__bootstrap=True, reg__bootstrap_features=True, reg__max_features=0.3, reg__max_samples=0.3, reg__n_estimators=50 
[CV]  reg__bootstrap=True, reg__bootstrap_features=True, reg__max_features=0.3, reg__max_samples=0.3, reg__n_estimators=50, total=   3.2s
[CV] reg__bootstrap=True, reg__bootstrap_features=True, reg__max_features=0.3, 

[Parallel(n_jobs=1)]: Done 432 out of 432 | elapsed: 64.1min finished


                                               model  Accuracy  PrecisionLab1  \
0  (StandardScaler(copy=True, with_mean=True, wit...  0.871892       0.871892   
1  (StandardScaler(copy=True, with_mean=True, wit...  0.871892       0.871892   

   RecallLab1  FMeasureLab1       AUC  
0         1.0      0.931562  0.796662  
1         1.0      0.931562  0.816064  


In [30]:
print(results)

                                               model  Accuracy  PrecisionLab1  \
0  (StandardScaler(copy=True, with_mean=True, wit...  0.871892       0.871892   
1  (StandardScaler(copy=True, with_mean=True, wit...  0.871892       0.871892   

   RecallLab1  FMeasureLab1       AUC  
0         1.0      0.931562  0.796662  
1         1.0      0.931562  0.816064  


In [36]:
results.to_csv("results.csv")